# Plotting Geostationary Operational Environmental Satellite (GOES) Data
**This notebook will plot true color satellite imagery from GOES-16 with an 550 nm aerosol optical depth (AOD) field overlain using the GOES-2-go package**
***

In [ ]:
pip install goes2go # install package

In [ ]:
# import packages and modules
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import pandas as pd
import numpy as np
from goes2go import GOES
from goes2go.data import goes_nearesttime
import matplotlib as mplt
import matplotlib.cm as cm
from datetime import datetime, timedelta

# Plot True Color Imagery and AOD

In [ ]:
# declare GOES-16 time and products
G16_true = GOES(satellite=16).nearesttime("2023-6-7 13")
G16_aod = GOES(satellite=16, product="ABI-L2-AODC").nearesttime("2023-6-7 13")

# list all available products
rgb_products = [i for i in dir(G16_true.rgb) if i[0].isupper()]
rgb_products

In [ ]:
# coordinates for extent
x = G16_aod['x'].values
y = G16_aod['y'].values

# aodlandmean = np.ma.masked_invalid(G16_aod['mean_aod550_land'].values)
aod = np.ma.masked_invalid(G16_aod['AOD'].values)

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = plt.axes(projection=ccrs.PlateCarree())
nc = getattr(G16_true.rgb, 'NaturalColor')()

# get bounds based on min and max values of lon and lat
lon_min = nc.x.min().values
lon_max = nc.x.max().values
lat_min = nc.y.min().values
lat_max = nc.y.max().values

# plot data
ax.imshow(nc, origin='upper', extent=[lon_min, lon_max, lat_min, lat_max]) # plot the true color field
mesh = ax.pcolormesh(x, y, aod, cmap='viridis', vmin=0, vmax=1, transform=ccrs.PlateCarree()) # plot the aod field
cbar = plt.colorbar(mesh, ax=ax, pad=0.04, orientation='horizontal') # set a colorbar
cbar.set_label('Aerosol Optical Depth at 550 nm',fontsize=14, fontweight='bold')
cbar.ax.tick_params(labelsize=13)
plt.title('GOES-16 True Color Imagery & \n 550 nm Aerosol Optical Depth at 2023-06-07 13:00 UTC', fontsize=16, fontweight='bold')
plt.savefig('GOES_true_aod.png', bbox_inches="tight", dpi=800)

## Run a loop to extract 24 hours of imagery

In [ ]:
dates = ['2023-6-7'] # set time period
hours6 = list(range(18, 24, 1)) # times of interest: every hour -- NOTE: consider computational expense while running this section of the notebook!
minutes = list(range(0, 60, 10)) # every 10 minutes -- NOTE: consider computational expense while running this section of the notebook!

In [ ]:
# plot for both days (6 and 7 June 2023) for certain hours every 10 minutes
for date in dates:
    if date == '2023-6-6':
        hours = hours6
    elif date == '2023-6-7':
        hours=hours7
        
    for hour in hours:
        for minute in minutes:
            # declare GOES-16 time and products
            G16_true = GOES(satellite=16).nearesttime(f'{date} {hour}:{minute}')
            G16_aod = GOES(satellite=16, product="ABI-L2-AODC").nearesttime(f'{date} {hour}:{minute}')
            # coordinates for extent
            x = G16_aod['x'].values
            y = G16_aod['y'].values
            
            aod = np.ma.masked_invalid(G16_aod['AOD'].values)
            fig = plt.figure(figsize=(10,8))
            ax = plt.axes(projection=ccrs.PlateCarree())
            # use imshow
            nc = getattr(G16_true.rgb, 'NaturalColor')()
            # Get geographic bounds
            lon_min = nc.x.min().values
            lon_max = nc.x.max().values
            lat_min = nc.y.min().values
            lat_max = nc.y.max().values
            
            ax.imshow(nc, origin='upper', extent=[lon_min, lon_max, lat_min, lat_max]) # plot true color field
            mesh = ax.pcolormesh(x, y, aod, cmap='viridis', vmin=0, vmax=1, transform=ccrs.PlateCarree()) # plot aod field
            
            cbar = plt.colorbar(mesh, ax=ax, pad=0.04, orientation='horizontal')
            cbar.set_label('Aerosol Optical Depth at 550 nm',fontsize=14, fontweight='bold')
            cbar.ax.tick_params(labelsize=13)
            if minute == 0:
                plt.title(f'GOES-16 True Color Imagery & \n 550 nm Aerosol Optical Depth at {date} {hour}:0{minute}Z', fontsize=16, fontweight='bold')
                plt.savefig(f'/home/jovyan/Desktop/Scripts/GOES_aodTrue/{date} {hour}:0{minute}_GOES_true_aod.png', bbox_inches="tight", dpi=800)
            else:
                plt.title(f'GOES-16 True Color Imagery & \n 550 nm Aerosol Optical Depth at {date} {hour}:{minute}Z', fontsize=16, fontweight='bold')
                plt.savefig(f'/home/jovyan/Desktop/Scripts/GOES_aodTrue/{date} {hour}:{minute}_GOES_true_aod.png', bbox_inches="tight", dpi=800)